In [1]:
import matplotlib.pyplot as plt
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import tensorflow as tf
import numpy as np
import glob
import os
import sys  
import cv2
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import preprocessing
import tensorflow as tf

from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [2]:
print(tf.__version__)
data_path="npz_data/ucf_classes/*"
img_size=(128,128,3)
frame_number=20
num_class=2



1.13.1


In [3]:
def load_data(data_path):
    data=np.empty([1,frame_number,img_size[0],img_size[1],3])
    label=np.empty([1,num_class])
    file_list=glob.glob(data_path)
    data_list=[d for d in file_list if "data" in d.split("/")[-1]]
    label_list=[l for l in file_list if "label" in l.split("/")[-1]]
#     for index in range(len(data_list)):
#         d=np.load(data_list[index])["arr_0"]
#         l=np.load(label_list[index])["arr_0"]
#         data=np.append(data,d,axis=0)
#         data=np.append(label,l,axis=0)
#     data=data[1:]
#     label=label[1:]
    for index in range(len(data_list)):
        data=np.load(data_list[index])["arr_0"]
        label=np.load(label_list[index])["arr_0"]
    return data,label

data,label=load_data(data_path)
print(data.shape)
print(label.shape)

(100, 20, 128, 128, 3)
(100, 2)


In [4]:
data = data.reshape(data.shape[0]*data.shape[1],img_size[0],img_size[1],3)
label = np.repeat(label[:, :], frame_number, axis=0)
print(label.shape)
print(data.shape)


(2000, 2)
(2000, 128, 128, 3)


In [5]:
train_x = data[:int(label.shape[0]*0.8),...]
train_y = label[:int(label.shape[0]*0.8),...]
valid_x = data[int(label.shape[0]*0.8):int(label.shape[0]*0.9),...]
valid_y = label[int(label.shape[0]*0.8):int(label.shape[0]*0.9),...]
test_x = data[int(label.shape[0]*0.9):,...]
test_y = label[int(label.shape[0]*0.9):,...]

In [6]:
print(test_y.shape)


(200, 2)


In [7]:
print(test_y)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 

In [8]:
import tensorflow_hub as hub


W0509 16:54:03.836516 139802550089472 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [9]:
learning_rate = 1e-6
X = tf.placeholder(tf.float32,[None,img_size[0],img_size[1],3])
labels = tf.placeholder(tf.float32,[None,num_class])
timesteps = 10
num_hidden = 128 
num_output = 64

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden,num_output]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_output]))
}

def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = tf.nn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

Instructions for updating:
Colocations handled automatically by placer.


W0509 16:54:03.854614 139802550089472 deprecation.py:323] From /home/jetherng/keras/lib/python3.5/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [11]:
module = hub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_035_128/feature_vector/2",trainable=False)
outputs = module(X)



dense1 = tf.layers.dense(inputs=outputs, units=num_hidden)
dense1 = tf.reshape(dense1,(-1,10,128))

lstm1 = RNN(dense1, weights, biases)

logits = tf.layers.dense(inputs=lstm1,units=num_class)


prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=labels))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
        
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
  

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0509 16:54:43.218725 139802550089472 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


W0509 16:54:43.315796 139802550089472 deprecation.py:323] From <ipython-input-9-20847fd1b905>:26: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


W0509 16:54:43.316759 139802550089472 deprecation.py:323] From <ipython-input-9-20847fd1b905>:29: static_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



W0509 16:54:43.409323 139802550089472 deprecation.py:323] From <ipython-input-11-b653360d46ef>:18: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [12]:
batch_size = 2
num_steps = int(train_x.shape[0]/batch_size)
display_step = 2

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(20):
        print("epoch {}".format(epoch))

        for step in range(0, num_steps):
            batch_x, batch_y = train_x[step:(step+1)*batch_size,...],train_y[step:(step+1)*batch_size,...]

                # Run optimization op (backprop)
            sess.run(train_op, feed_dict={X: batch_x, labels: batch_y})
            if step % display_step == 0 or step == 1:
                    # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                         labels: batch_y})
                print("Step " + str(step) + ", Minibatch Loss= " + \
                          "{:.4f}".format(loss) + ", Training Accuracy= " + \
                          "{:.3f}".format(acc))

    print("Optimization Finished!")

    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: valid_x,
                                          labels:valid_y}))
    [ps] = session.run(prediction,
                       feed_dict={X: valid_x,labels:valid_y})
#     print("Top 5 actions:")
#     for i in np.argsort(ps)[::-1][:5]:
#         print("%-22s %.2f%%" % (labels[i], ps[i] * 100))







epoch 0


InvalidArgumentError: Input to reshape is a tensor with 256 values, but the requested shape requires a multiple of 1280
	 [[node Reshape_1 (defined at <ipython-input-11-b653360d46ef>:7) ]]

Caused by op 'Reshape_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jetherng/keras/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jetherng/keras/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jetherng/keras/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/jetherng/keras/lib/python3.5/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.5/asyncio/base_events.py",